In [22]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from glob import glob
import plotly.graph_objects as go

import umap

In [23]:
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
dim_reducer = umap.UMAP(n_neighbors=15, n_components=2, metric="cosine")

In [51]:
path = "../results/copa/llama/CAPO/seed42/**/step_results.parquet"
df = pd.read_parquet(glob(path, recursive=True)[0])
prompts = df["prompt"].unique()
embs = embedder.encode(prompts)
embs_2d = dim_reducer.fit_transform(embs)
scores = df["score"].values

c:\Users\tzehl\anaconda3\envs\ds\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



In [52]:
fig = go.Figure(
    data=go.Contour(
        z=scores,
        x=embs_2d[:, 0],
        y=embs_2d[:, 1],
        colorscale="Viridis",
        contours=dict(
            coloring="heatmap",
            showlabels=True,  # show labels on contours
            labelfont=dict(  # label font properties
                size=12,
                color="white",
            ),
        ),
    )
).update_layout({"height": 800, "width": 800})

# show dots on top of the contour plot with color according to col step
fig.add_trace(
    go.Scatter(
        x=embs_2d[:, 0],
        y=embs_2d[:, 1],
        mode="markers",
        # marker=dict(
        #     size=8,
        #     color=df["step"],
        #     showscale=True
        # )
    )
)
fig.show()